In [1]:
import pandas as pd
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

  Using cached docx2txt-0.8-py3-none-any.whl
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [1]:
import os
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

GPT_API_KEY = os.environ["GPT_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
UPSTAGE_API_KEY = os.environ["UPSTAGE_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [10]:
folder_path = '../data'

In [21]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

CANCEL_main_qna_list.csv
DELIVERY_main_qna_list.csv
ORDER_main_qna_list.csv
REFUND_main_qna_list.csv
SUB_qna_list.csv
309


In [2]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [3]:
from langchain_pinecone import PineconeVectorStore


index_name = 'upstage-index'

In [24]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [4]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [17]:
query = '결제 후 주문 취소가 가능한가요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.498854] : 8
Question: Q[결제수단] 이미 주문했는데 결제 수단을 변경할 수 있나요?
Answer: 아니요. 상품을 주문/결제한 후에는 결제 수단 변경이 불가합니다. 상품이 아직 배송 전이라면 주문목록에서 주문을 취소한 후 원하는 결제 수단으로 다시 주문/결제해 주시기 바랍니다. 상품이 이미 배송을 시작했다면 배송 완료 후 반품을 접수해 주시기 바랍니다. 단 상품 배송 이후 단순 변심으로 반품할 경우 왕복 배송비가 부과될 수 있습니다.
 주문 취소하기
마이쿠팡 → 주문목록 → 상품선택 → [주문취소] 
이후 각 단계에 해당하는 항목을 선택하여 취소를 완료합니다.
 쿠페이 결제수단 관리하기
마이쿠팡 → 결제수단∙쿠페이 → [결제수단 관리] 
이후 원하는 결제수단을 등록 및 삭제합니다.
 단순 변심 반품 비용 (클릭)
keywords: ORDER
count: 0 [{'row': 8.0, 'source': '../data/ORDER_main_qna_list.csv'}]
* [SIM=0.498341] : 9
Question: Q[취소 철회] 주문취소 신청을 철회(취소)하고 싶어요.
Answer: 배송 상품은 주문을 취소하면 바로 결제 취소가 적용되어 이후 주문취소를 철회(취소) 할 수 없습니다. 이미 주문취소를 하셨다면 재구매를 부탁드립니다.
keywords: CANCEL
count: 0 [{'row': 9.0, 'source': '../data/CANCEL_main_qna_list.csv'}]
* [SIM=0.484310] : 44
Question: Q[취소] 신용카드로 결제한 주문을 취소했는데 카드대금이 청구되었습니다.
Answer: 신용카드로 결제한 주문을 취소하면 신용카드 승인 취소도 자동으로 진행됩니다. 그런데 만약 취소한 주문에 대해 카드대금 청구를 받으셨다면 이는 주문이 취소되기 전에 카드사에서 이미 청구서를 작성했기 때문입니다. 이 경우 청구된 카드대금이 취소되기까지는 특정 시간이 소요되며 이는 각 카드사마다 다릅니다. 

# LLM 질의 테스트

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

    

[(Document(id='e238a1b7-8a4a-414a-a98c-bd78492cd51b', metadata={'row': 71.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 71\nQuestion: Q[상품분실] 택배사에서 상품을 분실했다는데 어떻게 해야 하나요?\nAnswer: 택배사에서 분실한 상품에 대해서는 판매자 또는 택배사에 문의해 주시기 바랍니다. 분실한 상품의 재배송 및 환불은 판매자와 계약한 택배사의 확인 후 처리가 가능합니다.\n 상품 유형별 판매자 문의처\n• 로켓배송 로켓프레시 로켓모바일 로켓직구는 쿠팡 고객센터로 문의\n• 판매자 배송 상품은 아래의 경로를 통해 해당 판매자에게 직접 문의\n 판매자에 문의하기\n• 마이쿠팡 → 주문목록 → 상품선택 → [‘판매자명’에 문의하기]\n• 버튼 클릭 후 판매자 연락처 확인 및 문의 가능\n 택배사에 문의하기\n• 마이쿠팡 → 주문목록 → 상품선택 → [택배 배송기사에게 전화하기]\n• 택배기사 연락처 확인 불가 시 배송조회 후 택배사로 문의 가능\nkeywords: DELIVERY\ncount: 0'),
  0.493532956),
 (Document(id='e1b9146d-b5fd-4db5-9de2-75e44e18beaa', metadata={'row': 23.0, 'source': '../data/SUB_qna_list.csv'}, page_content='Question: 판매자 배송 상품을 받지 못한 경우\nAnswer: 택배사에 문의하기\n• 마이쿠팡 → 주문목록 → 상품선택 → [택배 배송기사에게 전화하기]\n• 택배기사 연락처는 배송완료 직후까지만 확인할 수 있습니다.\n 판매자에 문의하기\n• 마이쿠팡 → 주문목록 → 상품선택 → [‘판매자명’에 문의하기]\n• 상품 상세페이지 내 배송/교환/반품/판매자 안내에서 판매자 연락처 확인 가능\nkeywords: SUB\ncount: 0'),
  0.3913

In [19]:
# 유사도 보기
# results = database.similarity_search_with_score(query=query, k=3)
# pprint(results)
query = '결제 시 카드 정보는 어떻게 보호되나요?'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)

print()
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')

[(Document(id='4625c9d1-561e-488f-949d-6d3cff3d281f', metadata={'row': 67.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 67\nQuestion: Q[결제수단] 인터넷안전결제(ISP) 서비스란 무엇인가요?\nAnswer: 인터넷안전결제(ISP)란 카드 결제 시 카드번호를 직접 입력하지 않고 카드사에서 발급하는 가상의 인터넷안전결제(ISP) 비밀번호만으로 안전한 전자상거래를 할 수 있는 서비스입니다. 단 거래 금액이 30만원 이상일 때에는 공인 인증 과정이 필요합니다. 카드사별로 인터넷안전결제(ISP) 지원 여부가 상이하므로 자세한 내용은 사용하는 카드사로 문의해 주시기 바랍니다.\nkeywords: ORDER\ncount: 0'),
  0.377247423),
 (Document(id='3dc8309d-bc2c-4243-807e-09ba6d53c694', metadata={'row': 95.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 95\nQuestion: Q[결제] 토스카드/카카오페이 카드는 어떻게 결제하나요?\nAnswer: 신용/체크카드에서 ‘BC카드’를 선택한 후 [결제하기]를 클릭하면 카드 정보를 입력하여 결제할 수 있습니다.\n 단 토스유스카드는 일반 결제 시 ‘KB국민카드’로 선택하여 결제할 수 있습니다.\nkeywords: ORDER\ncount: 0'),
  0.363984585),
 (Document(id='656549d2-7b87-41d9-98be-90527f986d2c', metadata={'row': 66.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 66\nQuestion: Q[안심클릭] 안심클릭 서비스란 무엇인가요?\nAnswer: 인터넷 쇼핑 시 고객님의 카드에 미리

# OLLAMA TEST

In [65]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.
또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.

쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOllama(
    model="llama3.1:8b"
)

def merge_pages(pages):
    merged = "".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)
# 유사도 보기
# results = database.similarity_search_with_score(query=query, k=3)
# pprint(results)
query = '결제 후 주문 취소가 가능한가요?'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)

print()
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')
    

[(Document(id='afdbc8cb-ff9e-45e7-ac3c-c599f5c7ece1', metadata={'row': 8.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 8\nQuestion: Q[결제수단] 이미 주문했는데 결제 수단을 변경할 수 있나요?\nAnswer: 아니요. 상품을 주문/결제한 후에는 결제 수단 변경이 불가합니다. 상품이 아직 배송 전이라면 주문목록에서 주문을 취소한 후 원하는 결제 수단으로 다시 주문/결제해 주시기 바랍니다. 상품이 이미 배송을 시작했다면 배송 완료 후 반품을 접수해 주시기 바랍니다. 단 상품 배송 이후 단순 변심으로 반품할 경우 왕복 배송비가 부과될 수 있습니다.\n 주문 취소하기\n마이쿠팡 → 주문목록 → 상품선택 → [주문취소] \n이후 각 단계에 해당하는 항목을 선택하여 취소를 완료합니다.\n 쿠페이 결제수단 관리하기\n마이쿠팡 → 결제수단∙쿠페이 → [결제수단 관리] \n이후 원하는 결제수단을 등록 및 삭제합니다.\n 단순 변심 반품 비용 (클릭)\nkeywords: ORDER\ncount: 0'),
  0.498710096),
 (Document(id='87b51c87-546d-4efd-b32d-faf856d09f93', metadata={'row': 9.0, 'source': '../data/CANCEL_main_qna_list.csv'}, page_content=': 9\nQuestion: Q[취소 철회] 주문취소 신청을 철회(취소)하고 싶어요.\nAnswer: 배송 상품은 주문을 취소하면 바로 결제 취소가 적용되어 이후 주문취소를 철회(취소) 할 수 없습니다. 이미 주문취소를 하셨다면 재구매를 부탁드립니다.\nkeywords: CANCEL\ncount: 0'),
  0.49833414),
 (Document(id='92fba495-0129-4ec4-be5c-052016971ecf', metadata={'row': 44.0, 'so